In [1]:
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import pandas as pd
import copy
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize


Using TensorFlow backend.


In [2]:
with open('train-1.renju') as f:
    content = f.readlines()
data = [x.strip() for x in content] 

' '.join(list(filter(lambda a: 'p' not in a, data[34])))

f = open('renju-mod', 'w')
for elem in data:
    list(filter(lambda a: 'p' not in a, elem))
    f.write(' '.join(list(filter(lambda a: 'p' not in a, elem))) +'\n')  # python will convert \n to os.linesep
f.close()

In [3]:
def get_number(letter):
    #lst = {'a':0, 'b':1,}

    return ord(letter) - ord('a')

def get_pos(elem):
    w = get_number(elem[0])
    h = int(elem[1:]) - 1
    return w, h

In [4]:
test_data = data[int(0.7 * len(data)):]

In [5]:
data = data[: int(0.7 * len(data))]

In [6]:
import multiprocessing as mp
def parallel_game(_):
    elem = data[np.random.randint(len(data))]
    cur_pos = np.zeros((15,15, 3))
    game = elem.split()
    for iterator in range(1, len(game) - 1):
        elem = game[iterator]
        w, h = get_pos(elem)
        if iterator % 2 == 1:
            cur_pos[w][h][0] = 1 
        else:
            cur_pos[w][h][1] = 1


        for i in range(15):
            for j in range(15):
                if cur_pos[i][j][0] > 0:
                    cur_pos[i][j][2] += 1
                if cur_pos[i][j][1] > 0:
                    cur_pos[i][j][2] += 1


        cur_pos_to_app = copy.deepcopy(cur_pos)
        w, h = get_pos(game[iterator + 1])
        ans = h * 15 + w
        #print(cur_pos)
        return (cur_pos_to_app, ans)

In [6]:
from itertools import islice
def data_gen(data, test=False):
    part_size = len(data)//30
    while(1):
        neural_games = []
        neural_ans = []
        while(len(neural_games) < part_size):
            elem = data[np.random.randint(len(data))]
            cur_pos = np.zeros((15,15, 3))
            game = elem.split()
            for iterator in range(1, len(game) - 1):
                elem = game[iterator]
                w, h = get_pos(elem)
                if iterator % 2 == 1:
                    cur_pos[w][h][0] = 1 
                else:
                    cur_pos[w][h][1] = 1 
                
                cur_pos[:, :, 2:] = cur_pos.sum(2).reshape((15,15,1))
                        
                
                
                if iterator % 2 == 1:
                    cur_pos_to_app = copy.deepcopy(cur_pos)
                    w, h = get_pos(game[iterator + 1])
                    ans = h * 15 + w
                    #print(cur_pos)

                    neural_ans.append(ans)
                    neural_games.append(cur_pos_to_app)
                    if (len(neural_games) >= part_size):
                        break


        neural_games = np.array(neural_games)
        neural_ans = np.array(neural_ans)
        neural_ans = label_binarize(neural_ans, classes=[i for i in range(225)])
        neural_games.resize((len(neural_games), 1, 15,15,3))
        yield (neural_games, neural_ans)

In [65]:
model_policy.save("policy")

In [101]:
from itertools import islice
def data_gen_lr(data, test=False):
    part_size = len(data) // 4100
    while(1):
        neural_games = []
        neural_ans = []
        while(len(neural_games) < part_size):
            elem = data[np.random.randint(len(data))]
            cur_pos = np.zeros(225)
            game = elem.split()
            for iterator in range(1, len(game) - 1):
                elem = game[iterator]
                w, h = get_pos(elem)
                if iterator % 2 == 1:
                    cur_pos[h * 15 + w] = 1 
                else:
                    cur_pos[h * 15 + w] = 2 
                
                cur_pos_to_app = copy.deepcopy(cur_pos)
                w, h = get_pos(game[iterator + 1])
                ans = h * 15 + w
                #print(cur_pos)
                neural_ans.append(ans)
                neural_games.append(cur_pos_to_app)
                if (len(neural_games) >= part_size):
                    break


        neural_games = np.array(neural_games)
        neural_ans = np.array(neural_ans)
        #neural_ans = label_binarize(neural_ans, classes=[i for i in range(225)])
        #neural_games.resize((part_size, 1, 15,15,3))
        yield (neural_games, neural_ans)

In [107]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Ridge
#clf = LinearRegression(n_jobs=8)
#clf = KNeighborsClassifier(n_jobs = 4, n_neighbors=3)
clf = LogisticRegression(n_jobs = 8)
#clf = Ridge()

X, y = next(data_gen_lr(data))
clf.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=8,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [154]:
%%timeit
s = clf.predict([tester_1.lr_pos])[0]

21.7 µs ± 1.66 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [189]:
%%timeit
random.randint(0,224)

979 ns ± 10.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [190]:
%%timeit
np.random.randint(0,224)

1.26 µs ± 19.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [108]:
from sklearn.externals import joblib
filename = 'LinearClass'
_ = joblib.dump(clf, filename, compress=9)

In [7]:
from sklearn.externals import joblib
clf = joblib.load('LinearClass')

In [184]:
from sklearn.metrics import accuracy_score
X_test, y_true = next(data_gen_lr(test_data))
y_pred = clf.predict(X_test)
for i in range(len(y_pred)):
    y_pred[i] = int(y_pred[i])
    
""" 
for elem in clf.predict(X_test):
    if elem > int(elem) + 0.5:
        y_pred.append(int(elem) + 1)
    else:
        y_pred.append(int(elem))
"""        
accuracy_score(y_true, y_pred)

0.020689655172413793

In [19]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape, Convolution2D, Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import SGD, Adam
from keras.layers import GRU, LSTM
#from keras.regularizers import WeightRegularizer

model_policy = Sequential()

model_policy.add(Reshape((15,15,3), input_shape=(1,15,15,3)))
model_policy.add(Convolution2D(4, 5, 5, activation='relu'))
model_policy.add(BatchNormalization())
model_policy.add(Convolution2D(8, 5, 5, activation='relu'))
model_policy.add(Convolution2D(16, 3, 3, activation='relu'))
model_policy.add(BatchNormalization())
model_policy.add(Convolution2D(32, 3, 3, activation='relu'))

model_policy.add(Flatten(name='flatten'))


#model_policy.add(Dense(1100, activation='relu', name='fczero'))
model_policy.add(BayesianDense(output_dim=512, activation='relu'))
model_policy.add(BatchNormalization())
model_policy.add(BayesianDense(output_dim=256, activation='relu'))
#model_policy.add(Dropout(0.1))
model_policy.add(Dense(225, activation='softmax', name='fc2'))
model_policy.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model_policy.summary()

/usr/local/lib/python3.5/dist-packages/keras/engine/topology.py:379: UserWarning: The `regularizers` property of layers/models is deprecated. Regularization losses are now managed via the `losses` layer/model property.
  warnings.warn('The `regularizers` property of layers/models '


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
reshape_3 (Reshape)              (None, 15, 15, 3)     0           reshape_input_3[0][0]            
____________________________________________________________________________________________________
convolution2d_9 (Convolution2D)  (None, 11, 11, 4)     304         reshape_3[0][0]                  
____________________________________________________________________________________________________
batchnormalization_7 (BatchNorma (None, 11, 11, 4)     16          convolution2d_9[0][0]            
____________________________________________________________________________________________________
convolution2d_10 (Convolution2D) (None, 7, 7, 8)       808         batchnormalization_7[0][0]       
___________________________________________________________________________________________

In [ ]:
2 + 2

In [9]:
model_policy.load_weights("weights.11-0.283181.hdf5")

In [8]:
from keras.models import load_model
model_policy = load_model("zeros_policy_44")
model = model_policy

In [20]:
for iteration in range(1000):
    neural_games, neural_ans = next(data_gen(data))
    model_policy.fit(neural_games, neural_ans, batch_size=20096, nb_epoch=100, verbose=2,
                    callbacks=[model_saver, TQDMNotebookCallback()],
                     validation_data = next(data_gen(test_data, test=False)))

INFO (theano.gof.compilelock): Refreshing lock /home/axcel/.theano/compiledir_Linux-4.10--generic-x86_64-with-Ubuntu-17.04-zesty-x86_64-3.5.3-64/lock_dir/lock



Train on 46309 samples, validate on 19846 samples


Epoch 1/100


ValueError: rng_mrg gpu implementation does not support more than (2**31 -1) samples
Apply node that caused the error: GPU_mrg_uniform{CudaNdarrayType(float32, vector),inplace}(<CudaNdarrayType(float32, vector)>, MakeVector{dtype='int64'}.0)
Toposort index: 320
Inputs types: [CudaNdarrayType(float32, vector), TensorType(int64, vector)]
Inputs shapes: [(92160,), (1,)]
Inputs strides: [(1,), (8,)]
Inputs values: ['not shown', array([2963275776])]
Outputs clients: [['output'], [Shape_i{0}(GPU_mrg_uniform{CudaNdarrayType(float32, vector),inplace}.1), GpuSubtensor{:int64:}(GPU_mrg_uniform{CudaNdarrayType(float32, vector),inplace}.1, ScalarFromTensor.0), GpuSubtensor{int64::}(GPU_mrg_uniform{CudaNdarrayType(float32, vector),inplace}.1, ScalarFromTensor.0)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can

In [12]:
from keras.callbacks import ModelCheckpoint
from keras_tqdm import TQDMNotebookCallback, TQDMCallback
# keras, model definition...
model_saver = ModelCheckpoint("weights.{epoch:02d}-{val_acc:.6f}.hdf5",
                                              monitor='val_acc', verbose=0,
                                              save_best_only=True, save_weights_only=False,
                                              mode='auto')

""" 
tensorboard = keras.callbacks.TensorBoard(log_dir='./logs', 
                                          histogram_freq=0, 
                                          write_graph=True,
                                          write_images=True)
                                          """ 

" \ntensorboard = keras.callbacks.TensorBoard(log_dir='./logs', \n                                          histogram_freq=0, \n                                          write_graph=True,\n                                          write_images=True)\n                                          "

In [53]:
model_policy.save('zeros_policy_44')

In [ ]:
2 + 2

In [11]:
model_policy.fit_generator(data_gen(data, True),
        steps_per_epoch=500, epochs=350, verbose=0,
                   validation_data = data_gen(test_data, test=True), validation_steps=2,
                   callbacks=[model_saver, tensorboard, TQDMNotebookCallback()])

ResourceExhaustedError: OOM when allocating tensor with shape[72,2200]
	 [[Node: Variable_15/Assign = Assign[T=DT_FLOAT, _class=["loc:@Variable_15"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/gpu:0"](Variable_15, Const_21)]]

Caused by op 'Variable_15/Assign', defined at:
  File "/usr/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/axcel/.local/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/axcel/.local/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/axcel/.local/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/home/axcel/.local/lib/python3.5/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    s

In [85]:
model_policy.save("policy")

In [13]:
from keras.models import load_model
model_check_env = load_model("ReshapedModelsmall")

In [10]:
from keras.models import load_model, save_model

#save_model(model, "med_modelv1")

In [13]:
model_policy.load_weights("weights.04-0.400417.hdf5")

In [11]:
model = load_model("weights.11-0.283181.hdf5")

In [15]:
model_policy = load_model("ReshapedModel0.46-9")

In [16]:
model = load_model("Med5")

In [31]:
model.load_weights("weights.18-0.381543.hdf5")

In [9]:
from keras.models import load_model
from sklearn.externals import joblib


#clf = joblib.load('LinearClass')

#model = load_model('zeros_policy_44')

class RenjuTEST(object): 
    def __init__(self, player, mode):
        """
        player : 1 for black, 2 for white
        """
        self.cur_pos = np.zeros((15,15, 3))
        self.cur_player = 1
        self.player = player
        self.action_space = 225
        self.moves_done = 0
        self.ext_pos = np.zeros((25, 25))
        self.lr_pos = np.zeros(225)
        self.mode = mode
        self.obl_action = None
        self.win_action = None
        self.latest_move = (7,7)
        
        
    def in_step(self, action):
        """
        Run one timestep of the environment's dynamics. When end of episode
        is reached, reset() should be called to reset the environment's internal state.
        Input
        -----
        action : an action provided by the environment
        Outputs
        -------
        (observation, reward, done, info)
        observation : agent's observation of the current environment
        reward [Float] : amount of reward due to the previous action
        done : a boolean, indicating whether the episode has ended
        info : a dictionary containing other diagnostic information from the previous action
        """
        self.moves_done += 1
        if self.cur_player == 1:
            self.cur_pos[action % 15][action // 15][0] = 1
            self.lr_pos[action] = 1
        else:
            self.cur_pos[action % 15][action // 15][1] = 1
            self.lr_pos[action] = 2
            
            
            
        w = (action % 15) + 5
        h = (action // 15) + 5
        
        self.latest_move = (w - 5, h - 5)
        
        self.ext_pos[w][h] = self.cur_player
        
        """
        for i in range(15):
            for j in range(15):
                if self.cur_pos[i][j][0] > 0 or self.cur_pos[i][j][1] > 0:
                    self.cur_pos[i][j][2] += 1
        """
        
        
        self.cur_pos[:, :, 2:] = self.cur_pos.sum(2).reshape((15,15,1))
        
        reward = 0
        
        rdiag = [0, 0, 0]
        rdiaginv = [0, 0, 0]
        ldiag = [0, 0, 0]
        ldiaginv = [0, 0, 0]
        rrow = [0, 0, 0]
        lrow = [0, 0, 0]
        rcol = [0, 0, 0]
        lcol = [0, 0, 0]
        
        broken = [0,0,0,0,0,0,0,0]
        
        for i in range(5):
            
            if self.ext_pos[w + i][h + i]:
                if not broken[0]:
                    rdiag[int(self.ext_pos[w + i][h + i])] += 1
                else:
                    broken[0] = 1
                
            if self.ext_pos[w][h + i]:
                if not broken[1]:
                    rcol[int(self.ext_pos[w][h + i])] += 1
                else:
                    broken[1] = 1
                
            if self.ext_pos[w - i][h + i]:
                if not broken[2]:
                    ldiag[int(self.ext_pos[w - i][h + i])] += 1
                else:
                    broken[2] = 1
                
            
            if self.ext_pos[w - i][h]:
                if not broken[3]:
                    lrow[int(self.ext_pos[w - i][h])] += 1
                else:
                    broken[3] = 1
            
            if self.ext_pos[w - i][h - i]:
                if not broken[4]:
                    rdiaginv[int(self.ext_pos[w - i][h - i])] += 1
                else:
                    broken[4] = 1
            
            if self.ext_pos[w][h - i]:
                if not broken[5]:
                    lcol[int(self.ext_pos[w][h - i])] += 1
                else:
                    broken[5] = 1
            
            if self.ext_pos[w + i][h - i]:
                if not broken[6]:
                    ldiaginv[int(self.ext_pos[w + i][h - i])] += 1
                else:
                    broken[6] = 1
            
            if self.ext_pos[w + i][h]:
                if not broken[7]:
                    rrow[int(self.ext_pos[w + i][h])] += 1
                else:
                    broken[7] = 1
                    
        
        self.obl_action = None
        first_obl_action = None
        second_obl_action = None
        
        rightdiag = rdiag[self.cur_player] + rdiaginv[self.cur_player]
        
        if rightdiag >= 4:
            if rightdiag >= 6:
                if self.player == self.cur_player:
                    reward = 1
                else:
                    reward = -1
            else:
                if (self.ext_pos[w + rdiag[self.cur_player]][h + rdiag[self.cur_player]] == 0 and 
                 0 <= w - 5 + rdiag[self.cur_player] < 15 and 0 <= h - 5 + rdiag[self.cur_player] <= 15):
                    first_obl_action = (h - 5 + rdiag[self.cur_player]) * 15 + w - 5 + rdiag[self.cur_player]
                
                if (self.ext_pos[w - rdiaginv[self.cur_player]][h - rdiaginv[self.cur_player]] == 0 and 
                 0 <= w - 5 - rdiaginv[self.cur_player] < 15 and 0 <= h - 5 - rdiaginv[self.cur_player] <= 15):
                    second_obl_action = (h - 5 - rdiaginv[self.cur_player]) * 15 + w - 5 - rdiaginv[self.cur_player]
                
                if first_obl_action or second_obl_action:
                    if rightdiag == 5 and self.player == self.cur_player:
                        self.win_action = (first_obl_action, second_obl_action)
                    if first_obl_action:
                        self.obl_action = first_obl_action
                    else:
                        self.obl_action = second_obl_action
                    
                
                
            
        leftdiag = ldiag[self.cur_player] + ldiaginv[self.cur_player]
        
        if leftdiag >= 4:
            if leftdiag >= 6:
                if self.player == self.cur_player:
                    reward = 1
                else:
                    reward = -1
            else:
                if (self.ext_pos[w - ldiag[self.cur_player]][h + ldiag[self.cur_player]] == 0 and 
                 0 <= w - 5 - ldiag[self.cur_player] < 15 and 0 <= h - 5 + ldiag[self.cur_player] <= 15):
                    first_obl_action = (h - 5 + ldiag[self.cur_player]) * 15 + w - 5 - ldiag[self.cur_player]
                
                if (self.ext_pos[w + ldiaginv[self.cur_player]][h - ldiaginv[self.cur_player]] == 0 and 
                 0 <= w - 5 + ldiaginv[self.cur_player] < 15 and 0 <= h - 5 - ldiaginv[self.cur_player] <= 15):
                    second_obl_action = (h - 5 - ldiaginv[self.cur_player]) * 15 + w - 5 + ldiaginv[self.cur_player]
                
                if first_obl_action or second_obl_action:
                    if leftdiag == 5 and self.player == self.cur_player:
                        self.win_action = (first_obl_action, second_obl_action)
                    if first_obl_action:
                        self.obl_action = first_obl_action
                    else:
                        self.obl_action = second_obl_action
                    
        
        row = rrow[self.cur_player] + lrow[self.cur_player]
        
        if row >= 4:
            if row >= 6:
                if self.player == self.cur_player:
                    reward = 1
                else:
                    reward = -1
            else:
                if (self.ext_pos[w - lrow[self.cur_player]][h] == 0 and 
                 0 <= w - 5 - lrow[self.cur_player] < 15):
                    first_obl_action = (h - 5) * 15 + w - 5 - lrow[self.cur_player]
                
                if (self.ext_pos[w + rrow[self.cur_player]][h] == 0 and 
                 0 <= w - 5 + rrow[self.cur_player] < 15):
                    second_obl_action = (h - 5) * 15 + w - 5 + rrow[self.cur_player]
                
                if first_obl_action or second_obl_action:
                    if row == 5 and self.player == self.cur_player:
                        self.win_action = (first_obl_action, second_obl_action)
                    if first_obl_action:
                        self.obl_action = first_obl_action
                    else:
                        self.obl_action = second_obl_action
                    
        
        
        col = lcol[self.cur_player] + rcol[self.cur_player]
        
        if col >= 4:
            if col >= 6:
                if self.player == self.cur_player:
                    reward = 1
                else:
                    reward = -1
            else:
                if (self.ext_pos[w][h - lcol[self.cur_player]] == 0 and 
                 0 <= h - 5 - lcol[self.cur_player] <= 15):
                    first_obl_action = (h - 5 - lcol[self.cur_player]) * 15 + w - 5
                
                if (self.ext_pos[w][h + rcol[self.cur_player]] == 0 and 
                 0 <= h - 5 + rcol[self.cur_player] <= 15):
                    second_obl_action = (h - 5 + rcol[self.cur_player]) * 15 + w - 5
                
                if first_obl_action or second_obl_action:
                    if col == 5 and self.player == self.cur_player:
                        self.win_action = (first_obl_action, second_obl_action)
                        
                    if first_obl_action:
                        self.obl_action = first_obl_action
                    else:
                        self.obl_action = second_obl_action
                    
                    
        #if self.win_action != None:
        #    print("WARNING, WIN POSITION:", self.win_action)
            
        #print("pure", first_obl_action, second_obl_action)
        #print(self.win_action)
        
        if self.cur_player == 1:
            self.cur_player = 2
        else:
            self.cur_player = 1
        
        done = True if (self.moves_done == 225 or reward != 0) else False
        cur_pos = self.cur_pos
        if self.moves_done == 225:
            self.reset()
        info = dict()
        return (cur_pos, reward, done, info)
    
    def net_ans(self, model, mode = 'all'):
        s = model.predict(np.array([[self.cur_pos]]))[0]
        if mode == 'one':
            return np.argmax(s)
        else:
            #return sorted(range(len(s)), key=lambda k: s[k], reverse=True)
            return np.argsort(s)
    
    def simulation(self, model, mode):
        fake_env = RenjuTEST(1, 'neural')
        fake_env.cur_pos = np.copy(self.cur_pos)
        fake_env.cur_player = self.cur_player
        fake_env.player = self.player
        fake_env.action_space = self.action_space = 225
        fake_env.moves_done = self.moves_done
        fake_env.ext_pos = np.copy(self.ext_pos)
        fake_env.lr_pos = np.copy(self.lr_pos)
        fake_env.mode = self.mode
        reward = 0
        while reward == 0:
            """
            action = random.randint(0,224)
            while fake_env.cur_pos[action % 15][action // 15][0] != 0 or fake_env.cur_pos[action % 15][action // 15][1] != 0:
                action = random.randint(0,224)
                
            """ 
            #if mode == 'kn':
            s = clf.predict_proba([fake_env.lr_pos])[0]
            #else:
            #    s = model.predict(np.array([[fake_env.cur_pos]]))[0]
            action = np.argmax(s)
            if fake_env.cur_pos[action % 15][action // 15][0] != 0 or fake_env.cur_pos[action % 15][action // 15][1] != 0:
                net_move = np.argsort(s)[::-1]
                action = 0
                for act in net_move:
                    if fake_env.cur_pos[act % 15][act // 15][0] == 0 and fake_env.cur_pos[act % 15][act // 15][1] == 0:
                        action = act
                        break
            #"""
            cur_pos, reward, done, info = fake_env.in_step(action)
            
        if reward == 1:
            return 1
        else:
            return 0
    

    
    def step(self, action, mode = 'kn'):
        cur_pos, reward, done, info = self.in_step(action)
        
        if self.win_action != None:
            #print("WIN ACTION")
            if self.win_action[0]:
                if self.ext_pos[(self.win_action[0] % 15) + 5][(self.win_action[0] // 15) + 5] == 0:
                    action = self.win_action[0]
                    cur_pos, reward, done, info = self.in_step(self.win_action[0])
                    return cur_pos, reward, done, action
            if self.win_action[1]:
                if self.ext_pos[(self.win_action[1] % 15) + 5][(self.win_action[1] // 15) + 5] == 0:
                    action = self.win_action[1]
                    cur_pos, reward, done, info = self.in_step(self.win_action[1])
                    return cur_pos, reward, done, action
            
        
        if self.obl_action:
            action = self.obl_action
            cur_pos, reward, done, info = self.in_step(self.obl_action)
            return cur_pos, reward, done, action
        
        if reward != 0:
            #self.render()
            if done:
                self.reset()
            return cur_pos, reward, done, info
        else:
            if self.mode == 'neural':
                s = model.predict(np.array([[self.cur_pos]]))[0]
            else:
                s = clf.predict_proba([self.lr_pos])[0]
            #plt.figure()
            #k = s.reshape((15,15))
            #plt.imshow(k, cmap='hot', interpolation='nearest')
            #plt.show()
            action = np.argmax(s)
            if self.cur_pos[action % 15][action // 15][0] != 0 or self.cur_pos[action % 15][action // 15][1] != 0:
                net_move = np.argsort(s)[::-1]
                
                action = 0
                #print(net_move)
                for act in net_move:
                    if self.cur_pos[act % 15][act // 15][0] == 0 and self.cur_pos[act % 15][act // 15][1] == 0:
                        action = act
                        break
                #print("Net:", action)

            rnd = np.random.randint(1, 100)
            if rnd < -10:
                action = np.random.randint(0, 224)
                while self.cur_pos[action % 15][action // 15][0] != 0 or self.cur_pos[action % 15][action // 15][1] != 0:
                    action = np.random.randint(0, 224)
            print('Net action:', action)
            cur_pos, reward, done, info = self.in_step(action)
            return cur_pos, reward, done, action
    
    def learning(self, opponent):
        if self.moves_done % 2 == 0:
            s = opponent.predict(np.array([[self.cur_pos]]))[0]
        else:
            s = model.predict(np.array([[self.cur_pos]]))[0]
        action = np.argmax(s)
        if self.cur_pos[action % 15][action // 15][0] != 0 or self.cur_pos[action % 15][action // 15][1] != 0:
            net_move = np.argsort(s)[::-1]

            action = 0
            #print(net_move)
            for act in net_move:
                if self.cur_pos[act % 15][act // 15][0] == 0 and self.cur_pos[act % 15][act // 15][1] == 0:
                    action = act
                    break
            #print("Net:", action)
        rnd = np.random.randint(1, 100)
        if rnd < 5:
            action = np.random.randint(0, 224)
            while self.cur_pos[action % 15][action // 15][0] != 0 or self.cur_pos[action % 15][action // 15][1] != 0:
                action = np.random.randint(0, 224)

        cur_pos, reward, done, info = self.in_step(action)
        return cur_pos, reward, done, action
            
            
    
    def render(self, mode='human'):
        if mode == 'human':
            for j in reversed(range(15)):
                for i in range(15):
                    flag = 0
                    if self.cur_pos[i][j][0] == 1:
                        flag = 1
                        print("X", end=' ')
                    if self.cur_pos[i][j][1] == 1:
                        flag = 1
                        print("O", end=' ')
                    if not flag:
                        print("_", end=' ')
                print('\n', end='')
            print("------------------------------------------------\n")
        if mode == 'debug':
            for j in reversed(range(25)):
                for i in range(25):
                    flag = 0
                    if self.ext_pos[i][j] == 1:
                        flag = 1
                        print("X", end=' ')
                    if self.ext_pos[i][j] == 2:
                        flag = 1
                        print("O", end=' ')
                    if not flag:
                        print("_", end=' ')
                print('\n', end='')
            print("------------------------------------------------\n")
        
        
    def reset(self):
        """
        Resets the state of the environment, returning an initial observation.
        Outputs
        -------
        observation : the initial observation of the space. (Initial reward is assumed to be 0.)
        """
        self.cur_pos = np.zeros((15,15,3))
        self.cur_player = 1
        self.moves_done = 0
        self.ext_pos = np.zeros((25, 25))
        self.lr_pos = np.zeros(225)

        return self.cur_pos
    

In [14]:
model = model_policy

In [18]:
opponent = load_model('cross_30')

In [10]:
def look(cur_pos):
    for j in reversed(range(15)):
        for i in range(15):
            flag = 0
            if cur_pos[i][j] == 1:
                flag = 1
                print("X", end=' ')
            if not flag:
                print("_", end=' ')
        print('\n', end='')

In [99]:
import math
from copy import deepcopy
from time import time
from keras.models import load_model
from sklearn.externals import joblib


class MCSTNode():
    def __init__(self, env, len_nodes = 0):
        self.wins = 0
        self.all_games = 0
        self.childs = []
        self.env = env
        self.parent = None
        self.action = None
        self.index = len_nodes
        self.child_actions = []
        

class UCT():
    def __init__(self, env, model, mode):        
        self.root = MCSTNode(env)
        self.all_games = 0
        self.constant = 1.1
        self.cur_node = self.root
        self.model = model
        self.cur_pos = self.root
        self.mode = mode
        self.len_nodes = 1
        self.edges = []
        
    
    def get_child_UCT_list(self, node):
        return list(map(self.get_UCT_stat, node.childs))
        
    def explore(self, number):
        self.cur_node = self.root
        temp_look = np.zeros((15,15))
        if number == 0:
            print("EXPLORE ZERO NUM")
            overview = 4
            w, h = self.root.env.latest_move
            w_plus = min(15 - w, overview)
            w_minus = min(w, overview)
            
            h_plus = min(15 - h, overview)
            h_minus = min(h, overview)
            
            print("original", w, h)
            print("w-range", w_minus, w_plus)
            print("h-range", h_minus, h_plus)
            
            for j in range(h - h_minus, h + h_plus):
                for i in range(w - w_minus, w + w_plus):
                    if ((i,j) != (w,h)) and self.cur_node.env.ext_pos[i + 5][j + 5] == 0:
                        temp_look[i][j] = 1
                        new_node_action = (j * 15) + i
                        #print('new_node action', new_node_action)

                        new_env = deepcopy(self.cur_node.env)
                        new_env.cur_player = 1 if new_env.cur_player == 2 else 2
                        cur_pos, reward, done, action = new_env.in_step(new_node_action)
                        #print("REWARD NEW ROOT NODE", reward)
                        #new_env.render()

                        new_node = MCSTNode(new_env, self.len_nodes)
                        
                        if reward != 0:
                            new_node.wins = 999999999999999999
                            print("PIZDEC")
                        
                        new_node.env.cur_player = 1 if new_env.cur_player == 2 else 2

                        new_node.parent = self.cur_node
                        new_node.action = new_node_action
                        self.cur_node.child_actions.append(new_node_action)
                        self.len_nodes += 1

                        self.edges.append((self.cur_node.index, new_node.index))

                        run = new_node.env.simulation(self.model, self.mode)
                        self.all_games += 1

                        self.cur_node.childs.append(new_node)
                        self.cur_node = new_node

                        while(self.cur_node.parent != None):
                            self.cur_node.wins += run
                            self.cur_node.all_games += 1
                            self.cur_node = self.cur_node.parent
                        else:
                            self.cur_node.wins += run
                            self.cur_node.all_games += 1
            look(temp_look)                    
        UCT_list = self.get_child_UCT_list(self.cur_node)
        
        while len(UCT_list) > 0 and  max(UCT_list) > self.get_UCT_stat(self.cur_node):
            #print(UCT_list, "I am at ", self.cur_node.index)
            #print(UCT_list)
            best_node = np.argmax(UCT_list)
            self.cur_node = self.cur_node.childs[best_node]
            UCT_list = self.get_child_UCT_list(self.cur_node)
    
    def expand(self):
    
        
        #if self.mode == 'kn':
        #    s = clf.predict_proba([self.cur_node.env.lr_pos])[0]
        #else:
        s = model.predict(np.array([[self.cur_node.env.cur_pos]]))[0]
        action = np.argmax(s)
        if self.cur_node.env.cur_pos[action % 15][action // 15][0] != 0 or self.cur_node.env.cur_pos[action % 15][action // 15][1] != 0:
            net_move = np.argsort(s)[::-1]
            action = 0
            for act in net_move:
                if act not in self.cur_node.child_actions and self.cur_node.env.cur_pos[act % 15][act // 15][0] == 0 and self.cur_node.env.cur_pos[act % 15][act // 15][1] == 0:
                    action = act
                    break        

        new_node_action = action
        #print('New Node!')

        new_env = deepcopy(self.cur_node.env)
        cur_pos, reward, done, action = new_env.in_step(new_node_action)

        new_node = MCSTNode(new_env, self.len_nodes)

        if reward != 0:
            new_node.wins = 999999999999999999
        new_node.parent = self.cur_node
        new_node.action = new_node_action
        self.cur_node.child_actions.append(new_node_action)
        self.len_nodes += 1

        self.edges.append((self.cur_node.index, new_node.index))

        run = new_node.env.simulation(self.model, self.mode)
        self.all_games += 1

        self.cur_node.childs.append(new_node)
        self.cur_node = new_node

        while(self.cur_node.parent != None):
            self.cur_node.wins += run
            self.cur_node.all_games += 1
            self.cur_node = self.cur_node.parent
        else:
            self.cur_node.wins += run
            self.cur_node.all_games += 1

        """       
        else:
            run = self.cur_node.env.simulation(self.model, self.model)
            self.all_games += 1

            while(self.cur_node.parent != None):
                self.cur_node.wins += run
                self.cur_node.all_games += 1
                self.cur_node = self.cur_node.parent
            else:
                self.cur_node.wins += run
                self.cur_node.all_games += 1
        """  
            
            
    def search(self, time_limit):
        begin = time()
        point_action = np.zeros((225))
        cur_pos = self.root.env.cur_pos
        ext_pos = self.root.env.ext_pos
        cur_player = 1 if self.root.env.cur_player == 2 else 2
        """
        w,h = self.root.env.latest_move
        w += 5
        h += 5
        """ 
        
        for h_iter in range(15):
            for w_iter in range(15):
                w = w_iter + 5
                h = h_iter + 5
                rdiag = ""
                rdiaginv = ""
                ldiag = ""
                ldiaginv = ""
                rrow = ""
                lrow = ""
                rcol = ""
                lcol = ""


                rdiag_def = ""
                rdiaginv_def = ""
                ldiag_def = ""
                ldiaginv_def = ""
                rrow_def = ""
                lrow_def = ""
                rcol_def = ""
                lcol_def = ""


                rdiag_pos = []
                rdiaginv_pos = []
                ldiag_pos = []
                ldiaginv_pos = []
                rrow_pos = []
                lrow_pos = []
                rcol_pos = []
                lcol_pos = []

                rdiaginv_pos.append((w,h))
                ldiaginv_pos.append((w,h))
                lrow_pos.append((w,h))
                lcol_pos.append((w,h))


                for i in range(5):            
                    if ext_pos[w + i][h + i] == 0:
                        rdiag += "0"
                        rdiag_def += "0"
                    else:
                        if ext_pos[w + i][h + i] == cur_player:
                            rdiag += "1"
                            rdiag_def += "2"
                        else:
                            rdiag_def += "1"
                            rdiag += "2"
                    rdiag_pos.append((w + i, h + i))

                    if ext_pos[w][h + i] == 0:
                        rcol += "0"
                        rcol_def += "0"
                    else:
                        if ext_pos[w][h + i] == cur_player:
                            rcol += "1"
                            rcol_def += "2"
                        else:
                            rcol_def += "1"
                            rcol += "2"

                    rcol_pos.append((w, h + i))        

                    if ext_pos[w - i][h + i] == 0:
                        ldiag += "0"
                        ldiag_def += "0"
                    else:
                        if ext_pos[w - i][h + i] == cur_player:
                            ldiag += "1"
                            ldiag_def += "2"
                        else:
                            ldiag_def += "1"
                            ldiag += "2"
                    ldiag_pos.append((w - i, h + i))

                    if ext_pos[w - i][h] == 0:
                        lrow += "0"
                        lrow_def += "0"
                    else:
                        if ext_pos[w - i][h] == cur_player:
                            lrow += "1"
                            lrow_def += "2"
                        else:
                            lrow_def += "1"
                            lrow += "2"

                    lrow_pos.append((w - i, h))

                    if ext_pos[w - i][h - i] == 0:
                        rdiaginv += "0"
                        rdiaginv_def += "0"
                    else:
                        if ext_pos[w - i][h - i] == cur_player:
                            rdiaginv += "1"
                            rdiaginv_def += "2"
                        else:
                            rdiaginv_def += "1"
                            rdiaginv += "2"
                    rdiaginv_pos.append((w - i, h - i))

                    if ext_pos[w][h - i] == 0:
                        lcol += "0"
                        lcol_def += "0"
                    else:
                        if ext_pos[w][h - i] == cur_player:
                            lcol += "1"
                            lcol_def += "2"
                        else:
                            lcol_def += "1"
                            lcol += "2"

                    lcol_pos.append((w, h - i))

                    if ext_pos[w + i][h - i] == 0:
                        ldiaginv += "0"
                        ldiaginv_def += "0"
                    else:
                        if ext_pos[w + i][h - i] == cur_player:
                            ldiaginv += "1"
                            ldiaginv_def += "2"
                        else:
                            ldiaginv_def += "1"
                            ldiaginv += "2"

                    ldiaginv_pos.append((w + i, h - i))
                    if ext_pos[w + i][h] == 0:
                        rrow += "0"
                        rrow_def += "0"
                    else:
                        if ext_pos[w + i][h] == cur_player:
                            rrow += "1"
                            rrow_def += "2"
                        else:
                            rrow_def += "1"
                            rrow += "2"
                    rrow_pos.append((w + i, h))



                left_diag_full = ldiag[::-1] + ldiaginv[1:]
                left_diag_full_pos = ldiag_pos[::-1] + ldiaginv_pos[1:]
                left_diag_full_def = ldiag_def[::-1] + ldiaginv_def[1:]

                right_diag_full = rdiag[::-1] + rdiaginv[1:]
                right_diag_full_pos = rdiag_pos[::-1] + rdiaginv_pos[1:]
                right_diag_full_def = rdiag_def[::-1] + rdiaginv_def[1:]

                col_full = rcol[::-1] + lcol[1:]
                col_full_pos = rcol_pos[::-1] + lcol_pos[1:]
                col_full_def = rcol_def[::-1] + lcol_def[1:]


                row_full = lrow[::-1] + rrow[1:]
                row_full_pos = lrow_pos[::-1] + rrow_pos[1:]
                row_full_def = lrow_def[::-1] + rrow_def[1:]

                #print("left_diag", left_diag_full)
                #print("right_diag", right_diag_full)
                #print("column", col_full)
                #print("row", row_full)

                def find_occurances(what, where):
                    import re
                    return [m.start() for m in re.finditer(what, where)]

                pattern = [ 
                ['11111', 99999],
                ['011110', 7000], 
                ['01111', 4000], 
                ['11110', 4000],
                ['010111', 2000],
                ['011011', 2000],
                ['011101', 2000],
                ['111010', 2000],
                ['110110', 2000],
                ['101110', 2000],
                ['01110', 3000],
                ['0111', 1500],
                ['1110', 1500],
                ['01101', 800],
                ['01011', 800],
                ['11010', 800],
                ['10110', 800],
                ['0110', 200],
                ['10', 100],
                ['01', 100]
                ]


                comb_set = [[left_diag_full, left_diag_full_pos, left_diag_full_def], 
                            [right_diag_full, right_diag_full_pos, right_diag_full_def],
                            [col_full, col_full_pos, col_full_def],
                            [row_full, row_full_pos, row_full_def]]


                for match in pattern:
                    for comb in comb_set:
                        #print("I FOUND", match[0], "IN", comb[0], "AND", comb[2])
                        positions_attack = find_occurances(match[0], comb[0])
                        positions_def = find_occurances(match[0], comb[2])
                        #print("FOUND", positions_attack), positions_def

                        #print("pos_attack", positions_attack, match[0])
                        #print("pos_def", positions_def, match[0])

                        for left in positions_attack:
                            for i in range(left, left + len(match[0]) + 1):
                                #print("BONUSES AT", comb[1][i])
                                point_action[comb[1][i][0] - 5 + (comb[1][i][1] - 5) * 15] += match[1] 

                        for left in positions_def:
                            for i in range(left, left + len(match[0]) + 1):
                                point_action[comb[1][i][0] - 5 + (comb[1][i][1] - 5) * 15] += match[1] * 1.1

        for i in range(225):
            if ext_pos[(i % 15) + 5][(i // 15) + 5]:
                point_action[i] = -1
        
        for j in reversed(range(15)):
            for i in range(15):
                flag = 0
                if point_action[j * 15 + i] != 0:
                    flag = 1
                    print(point_action[j * 15 + i], end=' ')
                if not flag:
                    print("_", end=' ')
            print('\n', end='')

        
        number = 0
        while (time() - begin) < time_limit - (time() - begin) * 0.95:
            self.explore(number)
            self.expand()
            number += 1
        root_values = list(map(self.get_stat, self.root.childs))
        #print(root_values)
        #print(root_UCT_values, len(self.root.childs))
        if len(root_values) != 0:
            best_child = np.argmax(root_values)
        else:
            self.root = self.root.childs[0]
            return self.root.action
        print("I CHOOSE", self.root.childs[best_child].action)
        self.root = self.root.childs[best_child]
        
        if max(point_action) < 100:
            return self.root.action
        else:
            return np.argmax(point_action)
                    
    def get_UCT_stat(self, node):
        if node.all_games == 0:
            return 0
        else:
            return float(node.wins) / float(self.all_games) + self.constant * math.sqrt(math.log(self.all_games / node.all_games))
    
    def get_stat(self, node):
        if node.all_games == 0:
            return 0
        else:
            return float(node.wins) / float(self.all_games)
        

In [100]:

#Example program to illustrate my issue with dynamic buttons.
import tkinter
import random
from tkinter import *

class my_app(Frame):
    """Basic Frame"""
    def __init__(self, master, mode, time_for_search):
        self.buttons = []
        """Init the Frame"""
        Frame.__init__(self,master)
        self.grid()
        self.Create_Widgets()
        self.tester = RenjuTEST(2 if mode == 'me' else 1, mode)
        self.label = tkinter.Label(self, text='')
        self.label.grid(column=7, row=16, sticky=W);   #creates label for image on window 
        self.tree = UCT(self.tester, model, mode)
        self.mode = mode
        self.time_for_search = time_for_search


    def Create_Widgets(self):
        for i in range(15):
            for j in range(15): #Start creating buttons

                button_id = i * 15 + j 
                #print(self.button_id)

                self.newmessage = Button(self, #I want to bind the self.button_id to each button, so that it prints its number when clicked.
                                         text = '',
                                         anchor = W, command = lambda button_id=button_id: self.access(button_id))#Run the method

                #Placing
                self.newmessage.config(height = 1, width = 1)
                self.newmessage.grid(row = 15 - i, column = j, sticky = NW)
                self.buttons.append(self.newmessage)
        
    def access(self, b_id): #This is one of the areas where I need help. I want this to return the number of the button clicked.
        self.b_id = b_id
        print(b_id)
        self.buttons[b_id].config(text = 'X' if self.tester.cur_player == 1 else 'O')
        
        """ 
        cur_pos, reward, done, info = self.tester.step(b_id)
        self.tester.render(mode='human')
        
        self.buttons[info].config(text = 'X' if self.tester.cur_player == 2 else 'O')
        
        """ 
        cur_pos, reward, done, info = self.tester.in_step(b_id)
        tree_act_right = None
        if self.tester.obl_action:
            tree_act_right = self.tester.obl_action
            print("WARNING")
            
        if self.tester.win_action != None:
            print('Nyyyak')
            if self.tester.win_action[0]:
                if self.tester.ext_pos[(self.tester.win_action[0] % 15) + 5][(self.tester.win_action[0] // 15) + 5] == 0:
                    tree_act_right = self.tester.win_action[0]
            if self.tester.win_action[1]:
                if self.tester.ext_pos[(self.tester.win_action[1] % 15) + 5][(self.tester.win_action[1] // 15) + 5] == 0:
                    tree_act_right = self.tester.win_action[1]

        
        self.tester.render(mode='human')
        if reward != 0:
            ch = 'X' if self.tester.cur_player == 2 else 'O'
            self.label.config(text = str(ch + ' win'))
        
        
        """
        if b_id in self.tree.root.child_actions:
            for node in self.tree.root.childs:
                if node.action == b_id:
                    self.tree.root = node
                    break
        else:
        """
        self.tree = UCT(self.tester, model, self.mode)
        #tree = UCT(self.tester, model, 'kn' if random.randint(0,1) == 0 else 'neuron')
        self.tree.edges = []
        tree_act = self.tree.search(self.time_for_search)
        
        #print(tree_act, tree.root.all_games)
        #if tree_act_right:
        #    tree_act = tree_act_right
        self.buttons[tree_act].config(text = 'X' if self.tester.cur_player == 1 else 'O')
        print(self.tree.root.all_games, tree_act)
        cur_pos, reward, done, info = self.tester.in_step(tree_act)
        self.tester.render(mode='human')
        if reward != 0:
            ch = 'X' if self.tester.cur_player == 2 else 'O'
            self.label.config(text = str(ch + ' win'))
        #"""

In [18]:
model = model_policy

In [1]:
%load_ext Cython

In [102]:
import tkinter
import random
from tkinter import *


opponent = 'neural'
first_move = 'opp'
time_for_search = 0.5

edges = []

root = Tk()
root.title("Tic-Tac-Toe")
root.geometry("542x500")
app = my_app(root, first_move, time_for_search)


if first_move != 'me':
    model_policy = load_model('cross_40v4')
    tree_act = app.tree.search(time_for_search)
    #print(tree_act, tree.root.all_games)
    app.buttons[tree_act].config(text = 'X' if app.tester.cur_player == 1 else 'O')
    cur_pos, reward, done, info = app.tester.in_step(tree_act)

root.mainloop()

_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
EXPLORE ZERO NUM
original 7 7
w-range 4 4
h-range 4 4
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ X X X X X X X X _ _ _ _ 
_ _ _ X X X X X X X X _ _ _ _ 
_ _ _ X X X X X X X X _ _ _ _ 
_ _ _ X X X X _ X X X _ _ _ _ 
_ _ _ X X X X X X X X _ _ _ _ 
_ _ _ X X X X X X X X _ _ _ _ 
_ _ _ X X X X X X X X _ _ _ _ 
_ _ _ X X X X X X X X _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
I CHOOSE 48
112


_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
X X X X X X X _ _ _ _ _ _ _ _ 
X X X X X _ _ _ _ _ _ _ _ _ _ 
X X X X X X X _ _ _ _ _ _ _ _ 
X X X _ X X X _ _ _ _ _ _ _ _ 
X X X _ X X X _ _ _ _ _ _ _ _ 
X X X _ _ X X _ _ _ _ _ _ _ _ 
X X X X X X X _ _ _ _ _ _ _ _ 
X X X X X X X _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
I CHOOSE 15
2 64
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ O X O O _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ O _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ X X _ _ _ _ _ _ _ _ _ _ 
_ _ _ X X _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
------------------------------------------------

80


_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
X X X X _ _ _ _ _ _ _ _ _ _ _ 
X X X _ _ _ _ _ _ _ _ _ _ _ _ 
X _ _ _ _ X _ _ _ _ _ _ _ _ _ 
X X _ _ _ X _ _ _ _ _ _ _ _ _ 
X X X X X X _ _ _ _ _ _ _ _ _ 
X X X X X X _ _ _ _ _ _ _ _ _ 
X X X X X X _ _ _ _ _ _ _ _ _ 
I CHOOSE 0
2 127
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ X _ _ _ _ _ _ _ 
_ _ _ _ _ O X O O _ _ _ _ _ _ 
_ _ _ _ O X _ _ _ _ _ _ _ _ _ 
_ _ _ O X O _ _ _ _ _ _ _ _ _ 
_ O X X X _ _ _ _ _ _ _ _ _ _ 
_ _ O X X _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
------------------------------------------------



In [73]:
len(app.tree.edges)

255

In [49]:
def find_occurances(what, where):
    import re
    return [m.start() for m in re.finditer(what, where)]
    
find_occurances("0111", "011100111")

[0, 5]

In [127]:
import networkx as nx

g=nx.DiGraph()
g.add_edges_from(app.tree.edges)
p=nx.drawing.nx_pydot.to_pydot(g)
p.write_png('example.png')

True

![title](example.png)

In [27]:
%load_ext Cython

ImportError: No module named 'Cython'

In [ ]:
""" 
clf = joblib.load('LinearClass')

model = load_model('zeros_policy_44')

"""


In [98]:
clf = joblib.load('LinearClass')

In [13]:
%%cython
cimport numpy as np




import numpy as np
from keras.models import load_model
from sklearn.externals import joblib
import tkinter
import random
from tkinter import *
import math
from copy import deepcopy
from time import time
import numba


model = load_model('zeros_policy_44')
clf = joblib.load('LinearClass')

class RenjuTEST(): 
    def __init__(self, player, mode):
        """
        player : 1 for black, 2 for white
        """
        self.cur_pos = np.zeros((15,15, 3))
        self.cur_player = 1
        self.player = player
        self.action_space = 225
        self.moves_done = 0
        self.ext_pos = np.zeros((25, 25))
        self.lr_pos = np.zeros(225)
        self.mode = mode
        

    def in_step(self, action):
        """
        Run one timestep of the environment's dynamics. When end of episode
        is reached, reset() should be called to reset the environment's internal state.
        Input
        -----
        action : an action provided by the environment
        Outputs
        -------
        (observation, reward, done, info)
        observation : agent's observation of the current environment
        reward [Float] : amount of reward due to the previous action
        done : a boolean, indicating whether the episode has ended
        info : a dictionary containing other diagnostic information from the previous action
        """
        self.moves_done += 1
        if self.cur_player == 1:
            self.cur_pos[action % 15][action // 15][0] = 1
            self.lr_pos[action] = 1
        else:
            self.cur_pos[action % 15][action // 15][1] = 1
            self.lr_pos[action] = 2
            
            
        w = (action % 15) + 5
        h = (action // 15) + 5
        
        self.ext_pos[w][h] = self.cur_player
        
        """
        for i in range(15):
            for j in range(15):
                if self.cur_pos[i][j][0] > 0 or self.cur_pos[i][j][1] > 0:
                    self.cur_pos[i][j][2] += 1
        """
        self.cur_pos[:, :, 2:] = self.cur_pos.sum(2).reshape((15,15,1))

        reward = 0
        
        rdiag = [0, 0, 0]
        rdiaginv = [0, 0, 0]
        ldiag = [0, 0, 0]
        ldiaginv = [0, 0, 0]
        rrow = [0, 0, 0]
        lrow = [0, 0, 0]
        rcol = [0, 0, 0]
        lcol = [0, 0, 0]
        
        broken = [0,0,0,0,0,0,0,0]
        
        for i in range(5):
            
            if self.ext_pos[w + i][h + i]:
                if not broken[0]:
                    rdiag[int(self.ext_pos[w + i][h + i])] += 1
                else:
                    broken[0] = 1
                
            if self.ext_pos[w][h + i]:
                if not broken[1]:
                    rcol[int(self.ext_pos[w][h + i])] += 1
                else:
                    broken[1] = 1
                
            if self.ext_pos[w - i][h + i]:
                if not broken[2]:
                    ldiag[int(self.ext_pos[w - i][h + i])] += 1
                else:
                    broken[2] = 1
                
            
            if self.ext_pos[w - i][h]:
                if not broken[3]:
                    lrow[int(self.ext_pos[w - i][h])] += 1
                else:
                    broken[3] = 1
            
            if self.ext_pos[w - i][h - i]:
                if not broken[4]:
                    rdiaginv[int(self.ext_pos[w - i][h - i])] += 1
                else:
                    broken[4] = 1
            
            if self.ext_pos[w][h - i]:
                if not broken[5]:
                    lcol[int(self.ext_pos[w][h - i])] += 1
                else:
                    broken[5] = 1
            
            if self.ext_pos[w + i][h - i]:
                if not broken[6]:
                    ldiaginv[int(self.ext_pos[w + i][h - i])] += 1
                else:
                    broken[6] = 1
            
            if self.ext_pos[w + i][h]:
                if not broken[7]:
                    rrow[int(self.ext_pos[w + i][h])] += 1
                else:
                    broken[7] = 1
        
        
        if rdiag[self.cur_player] + rdiaginv[self.cur_player] >= 6:
            if self.player == self.cur_player:
                reward = 1
            else:
                reward = -1
                
        if ldiag[self.cur_player] + ldiaginv[self.cur_player] >= 6:
            if self.player == self.cur_player:
                reward = 1
            else:
                reward = -1
        
        if rrow[self.cur_player] + lrow[self.cur_player] >= 6:
            if self.player == self.cur_player:
                reward = 1
            else:
                reward = -1
                
        if lcol[self.cur_player] + rcol[self.cur_player] >= 6:
            if self.player == self.cur_player:
                reward = 1
            else:
                reward = -1
        """
        print('rdiag', rdiag)
        print('rdiaginv', rdiaginv)
        print('ldiag', ldiag)
        print('ldiaginv', ldiaginv)
        print('rrow', rrow)
        print('lrow', lrow)
        print('rcol', rcol)
        print('lcol', lcol)
        """
        
        if self.cur_player == 1:
            self.cur_player = 2
        else:
            self.cur_player = 1
        
        done = True if (self.moves_done == 225 or reward != 0) else False
        cur_pos = self.cur_pos
        if self.moves_done == 225:
            self.reset()
        info = dict()
        return (cur_pos, reward, done, info)
    
    def net_ans(self, model, mode = 'all'):
        s = model.predict(np.array([[self.cur_pos]]))[0]
        if mode == 'one':
            return np.argmax(s)
        else:
            #return sorted(range(len(s)), key=lambda k: s[k], reverse=True)
            return np.argsort(s)
    
    def simulation(self, model, mode):
        fake_env = RenjuTEST(1, 'neural')
        fake_env.cur_pos = np.copy(self.cur_pos)
        fake_env.cur_player = self.cur_player
        fake_env.player = self.player
        fake_env.action_space = self.action_space = 225
        fake_env.moves_done = self.moves_done
        fake_env.ext_pos = np.copy(self.ext_pos)
        fake_env.lr_pos = np.copy(self.lr_pos)
        fake_env.mode = self.mode
        reward = 0
        while reward == 0:
            
            action = random.randint(0,224)
            while fake_env.cur_pos[action % 15][action // 15][0] != 0 or fake_env.cur_pos[action % 15][action // 15][1] != 0:
                action = random.randint(0,224)
                
            """ 
            #if mode == 'kn':
            s = clf.predict_proba([fake_env.lr_pos])[0]
            #else:
            #    s = model.predict(np.array([[fake_env.cur_pos]]))[0]
            action = np.argmax(s)
            if fake_env.cur_pos[action % 15][action // 15][0] != 0 or fake_env.cur_pos[action % 15][action // 15][1] != 0:
                net_move = np.argsort(s)[::-1]
                action = 0
                for act in net_move:
                    if fake_env.cur_pos[act % 15][act // 15][0] == 0 and fake_env.cur_pos[act % 15][act // 15][1] == 0:
                        action = act
                        break
            """
            cur_pos, reward, done, info = fake_env.in_step(action)
            
        if reward == 1:
            return 1
        else:
            return 0
    


    def step(self, action, mode = 'kn'):
        cur_pos, reward, done, info = self.in_step(action)

        if reward != 0:
            #self.render()
            if done:
                self.reset()
            return cur_pos, reward, done, info
        else:
            if self.mode == 'neural':
                s = model.predict(np.array([[self.cur_pos]]))[0]
            else:
                s = clf.predict_proba([self.lr_pos])[0]
            #plt.figure()
            #k = s.reshape((15,15))
            #plt.imshow(k, cmap='hot', interpolation='nearest')
            #plt.show()
            action = np.argmax(s)
            if self.cur_pos[action % 15][action // 15][0] != 0 or self.cur_pos[action % 15][action // 15][1] != 0:
                net_move = np.argsort(s)[::-1]
                
                action = 0
                #print(net_move)
                for act in net_move:
                    if self.cur_pos[act % 15][act // 15][0] == 0 and self.cur_pos[act % 15][act // 15][1] == 0:
                        action = act
                        break
                #print("Net:", action)

            rnd = np.random.randint(1, 100)
            if rnd < -10:
                action = np.random.randint(0, 224)
                while self.cur_pos[action % 15][action // 15][0] != 0 or self.cur_pos[action % 15][action // 15][1] != 0:
                    action = np.random.randint(0, 224)
            print('Net action:', action)
            cur_pos, reward, done, info = self.in_step(action)
            return cur_pos, reward, done, action
    
    def learning(self, opponent):
        if self.moves_done % 2 == 0:
            s = opponent.predict(np.array([[self.cur_pos]]))[0]
        else:
            s = model.predict(np.array([[self.cur_pos]]))[0]
        action = np.argmax(s)
        if self.cur_pos[action % 15][action // 15][0] != 0 or self.cur_pos[action % 15][action // 15][1] != 0:
            net_move = np.argsort(s)[::-1]

            action = 0
            #print(net_move)
            for act in net_move:
                if self.cur_pos[act % 15][act // 15][0] == 0 and self.cur_pos[act % 15][act // 15][1] == 0:
                    action = act
                    break
            #print("Net:", action)
        rnd = np.random.randint(1, 100)
        if rnd < 5:
            action = np.random.randint(0, 224)
            while self.cur_pos[action % 15][action // 15][0] != 0 or self.cur_pos[action % 15][action // 15][1] != 0:
                action = np.random.randint(0, 224)

        cur_pos, reward, done, info = self.in_step(action)
        return cur_pos, reward, done, action
            
            
    
    def render(self, mode='human'):
        if mode == 'human':
            for j in reversed(range(15)):
                for i in range(15):
                    flag = 0
                    if self.cur_pos[i][j][0] == 1:
                        flag = 1
                        print("X", end=' ')
                    if self.cur_pos[i][j][1] == 1:
                        flag = 1
                        print("O", end=' ')
                    if not flag:
                        print("_", end=' ')
                print('\n', end='')
            print("------------------------------------------------\n")
        if mode == 'debug':
            for j in reversed(range(25)):
                for i in range(25):
                    flag = 0
                    if self.ext_pos[i][j] == 1:
                        flag = 1
                        print("X", end=' ')
                    if self.ext_pos[i][j] == 2:
                        flag = 1
                        print("O", end=' ')
                    if not flag:
                        print("_", end=' ')
                print('\n', end='')
            print("------------------------------------------------\n")
        
     
    def reset(self):
        """
        Resets the state of the environment, returning an initial observation.
        Outputs
        -------
        observation : the initial observation of the space. (Initial reward is assumed to be 0.)
        """
        self.cur_pos = np.zeros((15,15,3))
        self.cur_player = 1
        self.moves_done = 0
        self.ext_pos = np.zeros((25, 25))
        self.lr_pos = np.zeros(225)
        return self.cur_pos


class MCSTNode():
    def __init__(self, env):
        self.wins = 0
        self.all_games = 0
        self.childs = []
        self.env = env
        self.parent = None
        self.action = None
        self.index = np.random.randint(0, 10000000)
        self.child_actions = []
       

class UCT():
    def __init__(self, env, model, mode):        
        self.root = MCSTNode(env)
        self.all_games = 0
        self.constant = math.sqrt(2)
        self.cur_node = self.root
        self.model = model
        self.cur_pos = self.root
        self.mode = mode

    
    def get_child_UCT_list(self, node):
        return list(map(self.get_UCT_stat, node.childs))
       
    def explore(self):
        self.cur_node = self.root
        UCT_list = self.get_child_UCT_list(self.cur_node)
        while len(UCT_list) > 0 and  max(UCT_list) > self.get_UCT_stat(self.cur_node):
            #print(UCT_list, "I am at ", self.cur_node.index)
            #print(UCT_list)
            best_node = np.argmax(UCT_list)
            self.cur_node = self.cur_node.childs[best_node]
            UCT_list = self.get_child_UCT_list(self.cur_node)
    
    def expand(self):
        if self.cur_node.all_games > 1:
            #if self.mode == 'kn':
                #s = clf.predict_proba([self.cur_node.env.lr_pos])[0]
            #else:
            s = model.predict(np.array([[self.cur_node.env.cur_pos]]))[0]
            action = np.argmax(s)
            if self.cur_node.env.cur_pos[action % 15][action // 15][0] != 0 or self.cur_node.env.cur_pos[action % 15][action // 15][1] != 0:
                net_move = np.argsort(s)[::-1]
                action = 0
                for act in net_move:
                    if act not in self.cur_node.child_actions and self.cur_node.env.cur_pos[act % 15][act // 15][0] == 0 and self.cur_node.env.cur_pos[act % 15][act // 15][1] == 0:
                        action = act
                        break        

            new_node_action = action
            #print('New Node!')

            new_env = deepcopy(self.cur_node.env)
            new_env.in_step(new_node_action)

            new_node = MCSTNode(new_env)
            new_node.parent = self.cur_node
            new_node.action = new_node_action
            self.cur_node.child_actions.append(new_node_action)

            run = new_node.env.simulation(self.model, self.mode)
            self.all_games += 1

            self.cur_node.childs.append(new_node)
            self.cur_node = new_node

            while(self.cur_node.parent != None):
                self.cur_node.wins += run
                self.cur_node.all_games += 1
                self.cur_node = self.cur_node.parent
            else:
                self.cur_node.wins += run
                self.cur_node.all_games += 1
        else:
            run = self.cur_node.env.simulation(self.model, self.model)
            self.all_games += 1

            while(self.cur_node.parent != None):
                self.cur_node.wins += run
                self.cur_node.all_games += 1
                self.cur_node = self.cur_node.parent
            else:
                self.cur_node.wins += run
                self.cur_node.all_games += 1
            
            
    def search(self, time_limit):
        begin = time()
        while (time() - begin) < time_limit * 0.95:
            self.explore()
            self.expand()
        root_values = list(map(self.get_stat, self.root.childs))
        #print(root_UCT_values, len(self.root.childs))
        if len(root_values) != 0:
            best_child = np.argmax(root_values)
        else:
            self.root = self.root.childs[0]
            return self.root.action
        self.root = self.root.childs[best_child]
        return self.root.action
                    
    def get_UCT_stat(self, node):
        if node.all_games == 0:
            return 0
        else:
            return float(node.wins) / float(node.all_games) + self.constant * math.sqrt(math.log(self.all_games / node.all_games))
    
    def get_stat(self, node):
        if node.all_games == 0:
            return 0
        else:
            return float(node.wins) / float(node.all_games)


class my_app(Frame):
    """Basic Frame"""
    def __init__(self, master, mode, time_for_search):
        self.buttons = []
        """Init the Frame"""
        Frame.__init__(self,master)
        self.grid()
        self.Create_Widgets()
        self.tester = RenjuTEST(2 if mode == 'me' else 1, mode)
        self.label = tkinter.Label(self, text='')
        self.label.grid(column=7, row=16, sticky=W);   #creates label for image on window 
        self.tree = UCT(self.tester, model, mode)
        self.mode = mode
        self.time_for_search = time_for_search

    
    def Create_Widgets(self):
        for i in range(15):
            for j in range(15): #Start creating buttons

                button_id = i * 15 + j 
                #print(self.button_id)

                self.newmessage = Button(self, #I want to bind the self.button_id to each button, so that it prints its number when clicked.
                                         text = '',
                                         anchor = W, command = lambda button_id=button_id: self.access(button_id))#Run the method

                #Placing
                self.newmessage.config(height = 1, width = 1)
                self.newmessage.grid(row = 15 - i, column = j, sticky = NW)
                self.buttons.append(self.newmessage)
      
    def access(self, b_id): #This is one of the areas where I need help. I want this to return the number of the button clicked.
        self.b_id = b_id
        print(b_id)
        self.buttons[b_id].config(text = 'X' if self.tester.cur_player == 1 else 'O')
        
        """ 
        cur_pos, reward, done, info = self.tester.step(b_id)
        self.tester.render(mode='human')
        
        self.buttons[info].config(text = 'X' if self.tester.cur_player == 2 else 'O')
        
        #""" 
        cur_pos, reward, done, info = self.tester.in_step(b_id)
        
        #self.tester.render(mode='human')
        if reward != 0:
            ch = 'X' if self.tester.cur_player == 2 else 'O'
            self.label.config(text = str(ch + ' win'))
        
        if b_id in self.tree.root.child_actions:
            for node in self.tree.root.childs:
                if node.action == b_id:
                    self.tree.root = node
                    break
        else:
            self.tree = UCT(self.tester, model, self.mode)
        #tree = UCT(self.tester, model, 'kn' if random.randint(0,1) == 0 else 'neuron')
        tree_act = self.tree.search(self.time_for_search)
        print(self.tree.root.all_games, tree_act)
        #print(tree_act, tree.root.all_games)
        self.buttons[tree_act].config(text = 'X' if self.tester.cur_player == 1 else 'O')
        cur_pos, reward, done, info = self.tester.in_step(tree_act)
        #self.tester.render(mode='human')
        if reward != 0:
            ch = 'X' if self.tester.cur_player == 2 else 'O'
            self.label.config(text = str(ch + ' win'))
        #"""
        

opponent = 'neural'
first_move = 'me'
time_for_search = 10



root = Tk()
root.title("Tic-Tac-Toe")
root.geometry("542x500")
app = my_app(root, opponent, time_for_search)


if first_move != 'me':
    tree_act = app.tree.search(time_for_search)
    #print(tree_act, tree.root.all_games)
    app.buttons[tree_act].config(text = 'X' if app.tester.cur_player == 1 else 'O')
    cur_pos, reward, done, info = app.tester.in_step(tree_act)

root.mainloop()

112
4111 127
142
3906 129
157
4381 126


In [38]:
%%timeit
s = clf.predict([tester_1.lr_pos])[0]

62.7 µs ± 15.5 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [22]:
np.nonzero(tester_1.lr_pos)

(array([95]),)

In [37]:
%%timeit
s = model.predict(np.array([[tester_1.cur_pos]]))[0]

1.74 ms ± 184 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [16]:
from tqdm import tqdm

tester_1 = RenjuTEST(1, 'neural')
tester_2 = RenjuTEST(2, 'kn')

win = 0
loose = 0
all_game = 0

mode_1 = 'kn'
mode_2 = 'neuron'

tree_1 = UCT(tester_1, model, mode_1)
tree_2 = UCT(tester_2, model, mode_2)

for i in tqdm(range(150)):
    
    tree_act = tree_1.search(5)
    print(tree_act, tree_1.root.all_games)
    cur_pos, reward, done, info = tester_1.in_step(tree_act)
    tester_2.in_step(tree_act)
    
    if tree_act in tree_2.root.child_actions:
        for node in tree_2.root.childs:
            if node.action == tree_act:
                tree_2.root = node
                break
    else:
        tree_2 = UCT(tester_2, model, mode_2)
        
        
    if reward == 1:
        win += 1
        all_game += 1
    if reward == -1:
        loose += 1
        all_game += 1
        
    
    tree_act = tree_2.search(5)
    print(tree_act, tree_2.root.all_games)
    cur_pos, reward, done, info = tester_2.in_step(tree_act)
    tester_1.in_step(tree_act)
    if tree_act in tree_1.root.child_actions:
        for node in tree_1.root.childs:
            if node.action == tree_act:
                tree_1.root = node
                break
    else:
        tree_1 = UCT(tester_1, model, mode_1)
        
    if reward == -1:
        win += 1
        all_game += 1
    if reward == 1:
        loose += 1
        all_game += 1
        
    #tester.render(mode='human')

  0%|          | 0/150 [00:00<?, ?it/s]

95 2006


KeyboardInterrupt: 

In [364]:
print("win:", win, "loose:", loose, "all rate:", win / all_game)

win: 7 loose: 12 all rate: 0.3684210526315789


In [52]:
%lprun -f  tester.step tester.step(1)

In [32]:
indices

(array([0]), array([3]))

In [22]:
np.argmax(model_check_env.predict(np.array([[env.cur_pos]])))

205

In [55]:
def net_ans(env):
    s = model_check_env.predict(np.array([[env.cur_pos]]))[0]
    return sorted(range(len(s)), key=lambda k: s[k], reverse=True)

net_ans(env)

[205,
 157,
 79,
 191,
 63,
 202,
 22,
 175,
 111,
 91,
 189,
 152,
 142,
 34,
 25,
 160,
 72,
 33,
 204,
 36,
 176,
 19,
 57,
 48,
 137,
 178,
 133,
 52,
 136,
 106,
 151,
 132,
 77,
 187,
 102,
 103,
 92,
 26,
 118,
 88,
 20,
 42,
 109,
 35,
 67,
 172,
 121,
 126,
 78,
 93,
 119,
 169,
 61,
 39,
 177,
 153,
 71,
 24,
 148,
 59,
 201,
 18,
 162,
 107,
 161,
 104,
 139,
 122,
 146,
 64,
 62,
 190,
 127,
 186,
 194,
 199,
 41,
 167,
 198,
 74,
 166,
 56,
 124,
 40,
 215,
 131,
 134,
 174,
 159,
 164,
 51,
 182,
 87,
 117,
 37,
 140,
 110,
 155,
 185,
 168,
 76,
 54,
 171,
 154,
 7,
 206,
 49,
 193,
 149,
 73,
 90,
 123,
 163,
 31,
 147,
 183,
 192,
 138,
 108,
 58,
 89,
 4,
 170,
 65,
 69,
 116,
 70,
 94,
 145,
 66,
 47,
 50,
 184,
 200,
 44,
 21,
 5,
 2,
 27,
 86,
 125,
 80,
 105,
 97,
 101,
 209,
 141,
 156,
 220,
 179,
 181,
 55,
 17,
 75,
 208,
 43,
 219,
 216,
 30,
 14,
 96,
 6,
 223,
 46,
 120,
 150,
 32,
 12,
 213,
 135,
 212,
 217,
 95,
 195,
 222,
 15,
 3,
 196,
 10,
 214,
 13,

In [6]:
for i in range(len(model_normal.layers)):
    model.layers[i + 1].set_weights(model_normal.layers[i].get_weights())

In [17]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape, Convolution2D, Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import SGD, Adam
from keras.layers import GRU, LSTM

model_policy = Sequential()

model_policy.add(Reshape((15,15,3), input_shape=(1,15,15,3)))
model_policy.add(Convolution2D(8, (3,3), input_shape=(15, 15, 4), activation='relu'))

model_policy.add(Flatten(name='flatten'))


model_policy.add(Dense(550, activation='relu', name='fc1'))

model_policy.add(Dense(225, activation='softmax', name='fc2'))
model_policy.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model_policy.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_2 (Reshape)          (None, 15, 15, 3)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 13, 13, 8)         224       
_________________________________________________________________
flatten (Flatten)            (None, 1352)              0         
_________________________________________________________________
fc1 (Dense)                  (None, 550)               744150    
_________________________________________________________________
fc2 (Dense)                  (None, 225)               123975    
Total params: 868,349.0
Trainable params: 868,349.0
Non-trainable params: 0.0
_________________________________________________________________


In [19]:
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from keras.optimizers import Adam
import keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint
from rl.agents.sarsa import SarsaAgent


ENV_NAME = 'Renju'


# Get the environment and extract the number of actions.
env = RenjuTEST(1, 'kn')
np.random.seed(123)
nb_actions = 225

memory = SequentialMemory(limit=1000000, window_length=1)


policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05,
                              nb_steps=10000)


dqn = DQNAgent(model=model_policy, nb_actions=nb_actions, policy=policy, memory=memory,
                nb_steps_warmup=5000, gamma=.99, target_model_update=10000,
               train_interval=4, delta_clip=1.)
dqn.compile(Adam(lr=.00025), metrics=['mae'])


In [20]:
dqn.fit(env, nb_steps=175000, log_interval=10000, verbose = 2 )

# After training is done, we save the final weights one more time.
dqn.save_weights('latest_dqn', overwrite=True)

# Finally, evaluate our algorithm for 10 episodes.
dqn.test(env, nb_episodes=1, visualize=True)
dqn.test(env, nb_episodes=10, visualize=False)

Training for 175000 steps ...
     56/175000: episode: 1, duration: 0.512s, episode steps: 56, steps per second: 109, episode reward: -1.000, mean reward: -0.018 [-1.000, 0.000], mean action: 117.411 [3.000, 222.000], mean observation: 1.602 [0.000, 112.000], loss: --, mean_absolute_error: --, mean_q: --, mean_eps: --
     87/175000: episode: 2, duration: 0.150s, episode steps: 31, steps per second: 206, episode reward: 1.000, mean reward: 0.032 [0.000, 1.000], mean action: 129.194 [1.000, 219.000], mean observation: 0.564 [0.000, 61.000], loss: --, mean_absolute_error: --, mean_q: --, mean_eps: --
    124/175000: episode: 3, duration: 0.186s, episode steps: 37, steps per second: 199, episode reward: 1.000, mean reward: 0.027 [0.000, 1.000], mean action: 109.162 [3.000, 224.000], mean observation: 0.784 [0.000, 73.000], loss: --, mean_absolute_error: --, mean_q: --, mean_eps: --
    152/175000: episode: 4, duration: 0.213s, episode steps: 28, steps per second: 132, episode reward: 1.00

In [26]:
sarsa = SarsaAgent(model=model_policy, nb_actions=225)
sarsa.compile(Adam(), metrics=['mae'])

In [27]:
sarsa.load_weights("sarsa_Renju9_weights.h5f")

In [28]:
env = RenjuTEST(1, 'kn')

In [22]:
for epoch in range(10000):
    dqn.fit(env, nb_steps=17500, log_interval=10000, verbose = 0 )

    # After training is done, we save the final weights.
    dqn.save_weights('dqn', overwrite=True)

    # Finally, evaluate our algorithm for 5 episodes.
    dqn.test(env, nb_episodes=1, visualize=True)
    env = RenjuTEST(1, 'kn' if random.randint(0,1) == 0 else 'neural')
    dqn.test(env, nb_episodes=10, visualize=False)
    print('Opponent:', env.mode)

Testing for 1 episodes ...
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ O _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ X _ 
------------------------------------------------

_ _ _ _ _ _ _ _ _ _ O _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ O _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ 

KeyboardInterrupt: 